In [1]:
%matplotlib inline

import csv
import cv2
import math
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from random import shuffle
from skimage import exposure
from skimage import transform 

from sklearn.model_selection import train_test_split

def load_sample_list(paths, test_size=0.2):
  
    samples = []
    for path in paths:
        with open(path+'/driving_log.csv') as csvfile:
            reader = csv.reader(csvfile)
            next(reader)
            for line in reader:
                for i in range(3):
                    s = line[i].strip().replace('\\','/')
                    s = s.split('/')[-1]
                    line[i] = path + '/IMG/' + s
                samples.append(line)
    
    samples.pop(0)
    
    shuffle(samples)
    
    train_samples, validation_samples = train_test_split(samples, test_size=test_size) 
    
    return(train_samples, validation_samples)

def generate_samples(samples, batch_size=32, cameras=('l','r','c'), flip=True):
    
    expanded = []
    steering_adj = 0.2 
    
    if flip:
        flips = [False, True]
    else:
        flips = [False]        
    
    for sample in samples:
        for camera in cameras:
            for flip in flips:
                expanded.append((sample) + [camera, flip,])
    
    num_samples = len(expanded)
    
    while 1:
        shuffle(expanded)  
        for offset in range(0, num_samples, batch_size):
            x = []
            y = []            
            for batch_sample in expanded[offset:offset+batch_size]:
                center,left,right,steering,throttle,brake,speed,camera,flip = batch_sample
                steering, throttle, brake, speed = float(steering), float(throttle), float(brake), float(speed)
                
                steering *= 4
                
                if camera == 'c':
                    image = plt.imread(center)
                    steering = 0
                    
                if camera == 'l':                        
                    image = plt.imread(left)
                    steering += steering_adj                    
                    
                if camera == 'r':
                    image = plt.imread(right)
                    steering -= steering_adj
                 
                if flip:        
                    
                    image = np.fliplr(image) 
                    steering = -steering
                    
                x.append(image)
                y.append(steering)
                
            yield(np.array(x),np.array(y))

In [2]:
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Lambda, Convolution2D, Cropping2D, Dropout, BatchNormalization

train, valid = load_sample_list(['Track1','Track2'])

# compile and train the model using the generator function
n = 1
train_generator = generate_samples(train[::n], batch_size=32)
validation_generator = generate_samples(valid, batch_size=32)

rate = .5
    
model = Sequential()
model.add(Cropping2D(cropping=((70,25), (0,0)), input_shape=(160, 320, 3)))
model.add(BatchNormalization())

model.add(Convolution2D(24,5,5,subsample=(2,2),activation='elu', init='glorot_uniform', bias=False))
model.add(BatchNormalization())

model.add(Convolution2D(36,5,5,subsample=(2,2),activation='elu', init='glorot_uniform', bias=False))
model.add(BatchNormalization())

model.add(Convolution2D(48,5,5,subsample=(2,2),activation='elu', init='glorot_uniform', bias=False))
model.add(BatchNormalization())

model.add(Convolution2D(64,3,3,activation='elu', init='glorot_uniform', bias=False))
model.add(BatchNormalization())

model.add(Convolution2D(64,3,3,activation='elu', init='glorot_uniform', bias=False))
model.add(BatchNormalization())

model.add(Flatten())

bias = False
model.add(Dropout(rate))
model.add(Dense(1154, activation='elu', init='glorot_uniform', bias=bias))
model.add(BatchNormalization())

model.add(Dropout(rate))
model.add(Dense( 100, activation='elu',  init='glorot_uniform', bias=bias))
model.add(BatchNormalization())

#model.add(Dropout(rate))          
model.add(Dense(  50, activation='elu', init='glorot_uniform', bias=bias))
model.add(BatchNormalization())

#model.add(Dropout(rate))
model.add(Dense(  10, activation='elu', init='glorot_uniform', bias=bias))
model.add(BatchNormalization())

model.add(Dense(   1, activation='linear', init='glorot_uniform'))

model.compile(loss='mse', optimizer='adam')
expansion = 6
epochs = 2
model.fit_generator(train_generator, samples_per_epoch= \
            len(train)*expansion, validation_data=validation_generator, \
            nb_val_samples=len(valid)*expansion, nb_epoch=epochs)

model.save('model.h5')


Using TensorFlow backend.


Epoch 1/2
49692/49692 [==============================] - 100s - loss: 0.9846 - val_loss: 0.9349
Epoch 2/2
49692/49692 [==============================] - 96s - loss: 0.8845 - val_loss: 0.8272


# **Behavioral Cloning** 

## Writeup Template

### You can use this file as a template for your writeup if you want to submit it as a markdown file, but feel free to use some other method and submit a pdf if you prefer.

---

**Behavioral Cloning Project**

The goals / steps of this project are the following:
* Use the simulator to collect data of good driving behavior
* Build, a convolution neural network in Keras that predicts steering angles from images
* Train and validate the model with a training and validation set
* Test that the model successfully drives around track one without leaving the road
* Summarize the results with a written report

[//]: # (Image References)

[image1]: ./images/placeholder.png "Model Visualization"
[image2]: ./examples/placeholder.png "Grayscaling"
[image3]: ./examples/placeholder_small.png "Recovery Image"
[image4]: ./examples/placeholder_small.png "Recovery Image"
[image5]: ./examples/placeholder_small.png "Recovery Image"
[image6]: ./examples/placeholder_small.png "Normal Image"
[image7]: ./examples/placeholder_small.png "Flipped Image"

## Rubric Points
### Here I will consider the [rubric points](https://review.udacity.com/#!/rubrics/432/view) individually and describe how I addressed each point in my implementation.  

---
### Files Submitted & Code Quality

#### 1. Submission includes all required files and can be used to run the simulator in autonomous mode

My project includes the following files:
* model.py containing the script to create and train the model
* drive.py for driving the car in autonomous mode
* model.h5 containing a trained convolution neural network 
* writeup_report.md summarizing the results

#### 2. Submission includes functional code
Using the Udacity provided simulator and my drive.py file, the car can be driven autonomously around the track by executing 
```sh
python drive.py model.h5
```

#### 3. Submission code is usable and readable

The model.py file contains the code for training and saving the convolution neural network. The file shows the pipeline I used for training and validating the model, and it contains comments to explain how the code works.

### Model Architecture and Training Strategy

#### 1. An appropriate model architecture has been employed

My model consists of a convolution neural network with 3x3 filter sizes and depths between 32 and 128 (model.py lines 18-24) 

The model includes RELU layers to introduce nonlinearity (code line 20), and the data is normalized in the model using a Keras lambda layer (code line 18). 

#### 2. Attempts to reduce overfitting in the model

The model contains dropout layers in order to reduce overfitting (model.py lines 21). 

The model was trained and validated on different data sets to ensure that the model was not overfitting (code line 10-16). The model was tested by running it through the simulator and ensuring that the vehicle could stay on the track.

#### 3. Model parameter tuning

The model used an adam optimizer, so the learning rate was not tuned manually (model.py line 25).

#### 4. Appropriate training data

Training data was chosen to keep the vehicle driving on the road. I used a combination of center lane driving, recovering from the left and right sides of the road ... 

For details about how I created the training data, see the next section. 

### Model Architecture and Training Strategy

#### 1. Solution Design Approach

The overall strategy for deriving a model architecture was to ...

My first step was to use a convolution neural network model similar to the ... I thought this model might be appropriate because ...

In order to gauge how well the model was working, I split my image and steering angle data into a training and validation set. I found that my first model had a low mean squared error on the training set but a high mean squared error on the validation set. This implied that the model was overfitting. 

To combat the overfitting, I modified the model so that ...

Then I ... 

The final step was to run the simulator to see how well the car was driving around track one. There were a few spots where the vehicle fell off the track... to improve the driving behavior in these cases, I ....

At the end of the process, the vehicle is able to drive autonomously around the track without leaving the road.

#### 2. Final Model Architecture

The final model architecture (model.py lines 18-24) consisted of a convolution neural network with the following layers and layer sizes ...

Here is a visualization of the architecture (note: visualizing the architecture is optional according to the project rubric)

![alt text][image1]

#### 3. Creation of the Training Set & Training Process

To capture good driving behavior, I first recorded two laps on track one using center lane driving. Here is an example image of center lane driving:

![alt text][image2]

I then recorded the vehicle recovering from the left side and right sides of the road back to center so that the vehicle would learn to .... These images show what a recovery looks like starting from ... :

![alt text][image3]
![alt text][image4]
![alt text][image5]

Then I repeated this process on track two in order to get more data points.

To augment the data sat, I also flipped images and angles thinking that this would ... For example, here is an image that has then been flipped:

![alt text][image6]
![alt text][image7]

Etc ....

After the collection process, I had X number of data points. I then preprocessed this data by ...


I finally randomly shuffled the data set and put Y% of the data into a validation set. 

I used this training data for training the model. The validation set helped determine if the model was over or under fitting. The ideal number of epochs was Z as evidenced by ... I used an adam optimizer so that manually training the learning rate wasn't necessary.
